# 04 Detailed Preprocessing of Passive Data

This notebook shows the analysis of situational context using EMA and passive sensing data

1. **Load Data**: Load necessary data from pickle files.
2. **Preprocess EMA**:

In [1]:
import os
import sys
import regex as re
# If your current working directory is the notebooks directory, use this:
notebook_dir = os.getcwd()  # current working directory
src_path = os.path.abspath(os.path.join(notebook_dir, '..', 'src'))
sys.path.append(src_path)

# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
import glob
import pickle
from IPython.display import Markdown
from config import datapath, preprocessed_path

import pandas as pd
import numpy as np
import datetime as dt

from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from math import radians, cos, sin, asin, sqrt
import statistics  # Make sure this is imported


import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns 
import matplotlib.patches as mpatches

sns.set_context("notebook", rc={"axes.labelsize": 14, "xtick.labelsize": 14, "ytick.labelsize": 14})
sns.set_style("whitegrid", {'axes.grid': True})
%matplotlib inline
import plotly.express as px


In [2]:
backup_path = preprocessed_path + "backup_data_passive.feather"
df_backup = pd.read_feather(backup_path)

with open(preprocessed_path + '/ema_data.pkl', 'rb') as file:
    df_ema_framework = pickle.load(file)

with open(preprocessed_path + '/ema_content.pkl', 'rb') as file:
    df_ema_content = pickle.load(file)  

with open(preprocessed_path + '/monitoring_data.pkl', 'rb') as file:
    df_monitoring = pickle.load(file)

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Leonas_SSD/tiki_data/preprocessed/backup_data_passive.feather'

In [ ]:
# Configurations
# Check min. amount of EMA data available to map to passive data

timedelta_hours = 2
assess = 0

#GPS data
speed_limit = 1.4
max_distance = 150 
kms_per_radian = 6371.0088
epsilon = 0.03/kms_per_radian
min_samples = 30
min_nights_obs = 7
min_f_home = 0.5



In [ ]:
df_backup.startTimestamp.min()

## 1. Prepare passive features

In [ ]:
df_pass_act = df_backup.copy()

In [ ]:
# Only keep data that were collected during the first assessment phase
df_pass_act_base = df_pass_act[df_pass_act.startTimestamp <= df_pass_act.ema_base_end]

In [ ]:
df_pass_act.loc[df_pass_act.customer == "4MLe"]["startTimestamp"].min()

### 1.1 Calculate GPS features

In [ ]:
df_pass_act_loc =df_pass_act_base[df_pass_act_base.type.isin(["Latitude", "Longitude"])][["customer", "startTimestamp", "type", "doubleValue"]]

In [ ]:
df_loc = df_pass_act_loc.pivot_table(
    index=["customer", "startTimestamp"],
    columns="type",
    values=["doubleValue"],
    aggfunc='first'  # Using 'first' since each type should theoretically have only one entry per customer and timestamp
)

# Flatten the MultiIndex in columns
df_loc.columns = ['_'.join(col).strip() for col in df_loc.columns.values]

df_loc = df_loc.rename_axis(None, axis=1).reset_index()

# Rename the columns for clarity
df_loc = df_loc.rename(columns={
    'doubleValue_Latitude': 'Latitude',
    'doubleValue_Longitude': 'Longitude',
})

In [ ]:
class HomeClusterExtractor:
    def __init__(self, df, speed_limit, max_distance, epsilon, min_samples, min_nights_obs, min_f_home):
        self.df = df.copy()
        self.speed_limit = speed_limit
        self.max_distance = max_distance
        self.epsilon = epsilon
        self.min_samples = min_samples
        self.min_nights_obs = min_nights_obs
        self.min_f_home = min_f_home
        
        self.df['hour_gps'] = self.df['startTimestamp'].dt.hour
        self.df['day_gps'] = self.df['startTimestamp'].dt.date

    def calculate_distances_and_speeds(self):
        """Calculate distances and speeds for each customer."""
        self.df['distance'], self.df['time_diff'], self.df['speed'] = np.nan, np.nan, np.nan

        for customer in self.df['customer'].unique():
            mask = self.df['customer'] == customer
            customer_data = self.df.loc[mask]

            # Calculate distances between consecutive points
            distances = self._calculate_distances(customer_data)
            time_diffs = customer_data['startTimestamp'].diff().dt.total_seconds().fillna(0)
            speeds = distances / time_diffs.replace(0, np.nan)

            self.df.loc[mask, 'distance'] = distances
            self.df.loc[mask, 'time_diff'] = time_diffs
            self.df.loc[mask, 'speed'] = speeds

    def _calculate_distances(self, df):
        """Helper method to calculate distances using haversine formula."""
        coords = df[['Latitude', 'Longitude']].values
        distances = np.array([
            self._haversine(coords[i-1][1], coords[i-1][0], coords[i][1], coords[i][0])
            for i in range(1, len(coords))
        ])
        return np.append(distances, 0)  # Append 0 for the last point

    def _haversine(self, lon1, lat1, lon2, lat2):
        """Haversine formula to calculate distance between two lat/lon points in meters."""
        R = 6371000  # Radius of Earth in meters
        phi_1 = np.radians(lat1)
        phi_2 = np.radians(lat2)
        delta_phi = np.radians(lat2 - lat1)
        delta_lambda = np.radians(lon2 - lon1)
        a = np.sin(delta_phi/2.0)**2 + np.cos(phi_1) * np.cos(phi_2) * np.sin(delta_lambda/2.0)**2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        meters = R * c  # Output distance in meters
        return meters

    def extract_stationary_points(self):
        """Filter stationary points based on speed and distance."""
        return self.df[(self.df['speed'] < self.speed_limit) & (self.df['distance'] < self.max_distance)]

    def apply_clustering(self, df):
        """Apply DBSCAN clustering on stationary points."""
        return df.groupby('customer').apply(self._apply_dbscan).reset_index(drop=True)

    def _apply_dbscan(self, df):
        """Helper method to apply DBSCAN clustering."""
        clustering_model = DBSCAN(eps=self.epsilon, min_samples=self.min_samples, metric="haversine")
        cluster_labels = clustering_model.fit_predict(df[['Longitude', 'Latitude']].apply(np.radians))
        return pd.DataFrame({'cluster_100m': cluster_labels}, index=df.index)

    def find_home_cluster(self, geodata_clusters):
        """Identify the home cluster based on nighttime data."""
        # Filter for night hours
        geodata_night = geodata_clusters.loc[
            (geodata_clusters['hour_gps'] >= 22) | (geodata_clusters['hour_gps'] <= 6)
        ].copy()  # Make an explicit copy to avoid SettingWithCopyWarning

        # Calculate the mode of clusterID per user during night hours
        geodata_night['home'] = geodata_night.groupby('customer')['clusterID'].transform(
            lambda x: statistics.mode(x)
        )

        # Calculate various metrics to validate the home cluster
        geodata_night['nights_with_obs'] = geodata_night.groupby('customer')['day_gps'].transform('nunique')
        geodata_night['night_obs'] = geodata_night.groupby('customer')['day_gps'].transform('size')
        geodata_night['n_home'] = geodata_night.groupby('customer')['home'].transform(lambda x: x.value_counts().iloc[0])
        geodata_night['f_home'] = geodata_night['n_home'] / geodata_night['night_obs']

        # Update the 'home' label based on conditions
        geodata_night['home'] = geodata_night.apply(
            lambda x: x['home'] if x['nights_with_obs'] >= self.min_nights_obs and x['f_home'] > self.min_f_home else None, axis=1
        )

        # Extract a mapping of userID to home cluster
        user_home_mapping = geodata_night[['customer', 'home']].drop_duplicates()

        # Merging back to the full dataset
        return pd.merge(geodata_clusters, user_home_mapping, on='customer', how='left')


    def run(self):
        """Run the full extraction process."""
        self.calculate_distances_and_speeds()
        stationary_df = self.extract_stationary_points()
        geodata_cluster_df = self.apply_clustering(stationary_df)

        geodata_clusters = pd.concat([stationary_df.reset_index(drop=True), geodata_cluster_df['cluster_100m']], axis=1)
        geodata_clusters['clusterID'] = geodata_clusters['customer'].astype(str) + '00' + geodata_clusters['cluster_100m'].astype(str)

        return self.find_home_cluster(geodata_clusters)

# Example usage:
extractor = HomeClusterExtractor(df_loc, speed_limit=speed_limit, max_distance=max_distance, epsilon=epsilon, min_samples=min_samples, 
                                 min_nights_obs=min_nights_obs, min_f_home=min_f_home)
home_clusters = extractor.run()


In [ ]:
home_clusters.head()

## 2. Prepare EMA data

In [ ]:
df_ema_udi = df_ema_content[["customer", "createdAt_day", "quest_create", "unique_day_id", "assess"]]

In [ ]:
# Group by customer and unique_day_id and calculate the minimum quest_create
df_min_quest = df_ema_udi.groupby(['customer', 'unique_day_id'])['quest_create'].min().reset_index()

# Rename the column to sensor_block_end
df_min_quest.rename(columns={'quest_create': 'sensor_block_end'}, inplace=True)

# Merge the minimum quest_create back to the original DataFrame
df_ema_udi = pd.merge(df_ema_udi, df_min_quest, on=['customer', 'unique_day_id'], how='left')

# Create the sensor_block_start column, which is 2 hours before quest_create
df_ema_udi.drop(columns=['quest_create'], inplace=True)
df_ema_udi = df_ema_udi.drop_duplicates()

In [ ]:
df_ema_udi['sensor_block_start'] = df_ema_udi['sensor_block_end'] - pd.Timedelta(hours=2)


In [ ]:
# Only include first assessment phase
df_ema_udi_base = df_ema_udi.loc[df_ema_udi.assess == 0]

## 3. Merge EMA to passive data

In [ ]:
import pandas as pd
import numpy as np

class EMAMapper:
    def __init__(self, df_ema, df_data):
        self.df_ema = df_ema.copy()
        self.df_data = df_data.copy()

        self.df_ema['sensor_block_start'] = pd.to_datetime(self.df_ema['sensor_block_start'])
        self.df_ema['sensor_block_end'] = pd.to_datetime(self.df_ema['sensor_block_end'])
        self.df_data['startTimestamp'] = pd.to_datetime(self.df_data['startTimestamp'])
        if 'endTimestamp' in self.df_data.columns:
            self.df_data['endTimestamp'] = pd.to_datetime(self.df_data['endTimestamp'])

        self.outlier_distances = []  # To collect filtered outlier distances

    def _haversine(self, lon1, lat1, lon2, lat2):
        lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
        c = 2 * np.arcsin(np.sqrt(a))
        r = 6371  # Radius of Earth in kilometers
        distance = c * r

        return distance

    def count_gps_rows_within_blocks(self, max_distance_km=50):
        """
        Counts the number of GPS rows (n_GPS) where startTimestamp lies between sensor_block_start and sensor_block_end.
        Also calculates the total distance within each block using the Haversine formula and the average speed.
        Filters out unrealistic distances.

        Parameters:
        - max_distance_km: Maximum realistic distance between two consecutive GPS points. Distances above this value will be excluded.

        Returns:
        - df_ema with additional columns 'n_GPS', 'total_distance_km', and 'avg_speed_kmph' containing the count of GPS rows, total distance, and average speed within each block.
        """
        gps_counts = []
        total_distances = []
        avg_speeds = []

        df_gps = self.df_data[self.df_data['type'].isin(['Latitude', 'Longitude'])]
        df_gps = df_gps.pivot_table(index=["customer", "startTimestamp"], columns="type", values="doubleValue", aggfunc='first').reset_index()

        for idx, ema_row in self.df_ema.iterrows():
            sensor_block_start = ema_row['sensor_block_start']
            sensor_block_end = ema_row['sensor_block_end']

            df_filtered = df_gps[(df_gps['startTimestamp'] >= sensor_block_start) & 
                                 (df_gps['startTimestamp'] <= sensor_block_end)]

            if df_filtered.empty:
                gps_counts.append(0)
                total_distances.append(0)
                avg_speeds.append(0)
            else:
                df_filtered = df_filtered[(df_filtered['Latitude'] != 0) & (df_filtered['Longitude'] != 0)]
                df_filtered = df_filtered.dropna(subset=['Latitude', 'Longitude'])

                gps_count = df_filtered.shape[0]
                gps_counts.append(gps_count)

                latitudes = df_filtered['Latitude'].values
                longitudes = df_filtered['Longitude'].values
                timestamps = df_filtered['startTimestamp'].values

                distances = []
                speeds = []
                for i in range(1, len(latitudes)):
                    distance = self._haversine(longitudes[i-1], latitudes[i-1], longitudes[i], latitudes[i])

                    # Calculate time difference in hours between consecutive points
                    time_diff_hours = (timestamps[i] - timestamps[i-1]).astype('timedelta64[s]').astype(float) / 3600

                    if distance > max_distance_km:
                        self.outlier_distances.append({
                            'customer': df_filtered.iloc[i]['customer'],
                            'unique_day_id': ema_row['unique_day_id'],
                            'segment': i,
                            'distance_km': distance
                        })
                        distances.append(0)  # Set outlier distances to 0
                        speeds.append(0)  # Set outlier speeds to 0
                    else:
                        distances.append(distance)
                        if time_diff_hours > 0:
                            speeds.append(distance / time_diff_hours)
                        else:
                            speeds.append(0)

                total_distance = np.sum(distances)
                avg_speed = np.mean(speeds) if len(speeds) > 0 else 0

                total_distances.append(total_distance)
                avg_speeds.append(avg_speed)

        self.df_ema['n_GPS'] = gps_counts
        self.df_ema['total_distance_km'] = total_distances
        self.df_ema['avg_speed_kmph'] = avg_speeds

        return self.df_ema

    def map_steps_to_ema(self, max_steps_km=50):
        """
        Maps Steps data to EMA blocks based on time overlap and calculates n_steps.
        Rounds n_steps to the nearest whole number and fills with 0 if no Steps data is found within the block.
        Filters out unrealistic step values.

        Parameters:
        - max_steps_km: Maximum realistic distance that steps data should correspond to. Values above this will be considered outliers and excluded.

        Returns:
        - df_ema with an additional column 'n_steps' containing the summed and rounded Steps values.
        """
        n_steps_values = []

        df_steps = self.df_data[self.df_data['type'] == 'Steps']

        for idx, ema_row in self.df_ema.iterrows():
            sensor_block_start = ema_row['sensor_block_start']
            sensor_block_end = ema_row['sensor_block_end']

            df_filtered = df_steps[(df_steps['startTimestamp'] < sensor_block_end) & 
                                   (df_steps['endTimestamp'] > sensor_block_start)]

            if df_filtered.empty:
                n_steps_values.append(0)
            else:
                overlap_start = df_filtered['startTimestamp'].combine(sensor_block_start, max)
                overlap_end = df_filtered['endTimestamp'].combine(sensor_block_end, min)

                overlap_duration = (overlap_end - overlap_start).dt.total_seconds()
                step_duration = (df_filtered['endTimestamp'] - df_filtered['startTimestamp']).dt.total_seconds()

                proportion = overlap_duration / step_duration
                weighted_value = proportion * df_filtered['doubleValue']

                n_steps = weighted_value.sum()
                n_steps_values.append(round(n_steps))

        self.df_ema['n_steps'] = n_steps_values

        return self.df_ema


In [ ]:
# Initialize the EMAMapper class
ema_mapper = EMAMapper(df_ema_udi_base, df_pass_act_base)

# Map passive steps to EMA blocks
df_ema_with_mapped_values = ema_mapper.map_steps_to_ema()

# Count GPS rows within EMA blocks
df_ema_with_gps_counts = ema_mapper.count_gps_rows_within_blocks()

In [ ]:
df_empty= df_ema_with_gps_counts.loc[(df_ema_with_gps_counts.n_GPS == 0) &(df_ema_with_gps_counts.n_steps == 0) ]